In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp30opz1l6/pubring.gpg' created
gpg: /tmp/tmp30opz1l6/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os

In [0]:
  os.chdir("drive/app")

In [6]:
!ls

colorization_model.ipynb  outputs  test  TestColabNotebook.ipynb  train


In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
#import os
import re
import sys
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.misc import imsave 
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
IMAGE_SIZE = 224
NUM_EPOCHS = 500
EVAL_FREQUENCY = 1
#FLAGS = tf.app.flags.FLAGS

batch_size = 70

In [0]:
INITIAL_LEARNING_RATE = 0.001       # Initial learning rate.
num = 0

In [0]:
#def data_type():
 # if FLAGS.use_fp16:
 #   return tf.float16
 # else:
 #   return tf.float32

In [0]:
def display_images(color_imgs):
    global num
    color_imgs_shape = np.shape(color_imgs)
    print("color imgs shape shape " + str(color_imgs_shape))
    for i in range(color_imgs_shape[0]):
       # print("i is " + str(i))
       
       num += 1
       imsave('outputs/generatedImg' + str(num) + '.jpg', color_imgs[i])
       if num >= 70:
          num = 0
       #cv2.imshow('image',color_imgs[i])
       #cv2.waitKey(0) 
    return

In [0]:
def read_img(filename):
    print("filename " + str(filename))
    img = cv2.imread(filename)
    height, width, channels = img.shape
   # greyimg = cv2.cvtColor(cv2.resize(img, (224, 224)),cv2.COLOR_BGR2GRAY)
    colorimg = cv2.cvtColor(cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE)),cv2.COLOR_BGR2LAB)
    colorimg = colorimg.astype(np.float)/255.
    #return greyimg, colorimg
    return colorimg

In [0]:
def create(dirname):
    filenames = os.listdir(dirname)
    print("dirname " + str(dirname))
    x_img = []
    y_img = []
    for filename in filenames:
        #grey, color = read_img(dirname+filename)
        #grey_3 = np.atleast_3d(grey)
        color = read_img(dirname + filename)
        color_l = color[:,:,0]
        color_l3 = np.atleast_3d(color_l)
        color_ab = color[:,:,1:]
        x_img.append(color_l3)
        y_img.append(color_ab)
        
    x_img = np.asarray(x_img)
    y_img = np.asarray(y_img)
    return x_img, y_img

In [0]:
def create_network(image):
	# low_level_conv1
    with tf.variable_scope('low_level_conv1') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 1, 64], stddev=5e-2))
        bias = tf.Variable(tf.constant(0.1, shape=[64]))
        conv = tf.nn.conv2d(image, weight, [1, 2, 2, 1], padding="SAME")
        low_level_conv1 = tf.nn.relu(tf.nn.bias_add(conv, bias))
#        _activation_summary(low_level_conv1)

	# low_level_conv2
    with tf.variable_scope('low_level_conv2') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128],stddev=5e-2))
        bias = tf.Variable(tf.constant(0.1, shape=[128]))
        conv = tf.nn.conv2d(low_level_conv1, weight, [1, 1, 1, 1], padding="SAME")
        low_level_conv2 = tf.nn.relu(tf.nn.bias_add(conv, bias))
#        _activation_summary(low_level_conv2)
    
	# low_level_conv3
    with tf.variable_scope('low_level_conv3') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
        biases = tf.Variable(tf.constant(0.1, shape=[128]))
        conv = tf.nn.conv2d(low_level_conv2, weight, [1, 2, 2, 1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        low_level_conv3 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(low_level_conv3)

	# low_level_conv4
    with tf.variable_scope('low_level_conv4') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 256], stddev=5e-2))
        biases = tf.Variable(tf.constant(0.1, shape=[256]))
        conv = tf.nn.conv2d(low_level_conv3, weight, [1, 1, 1, 1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        low_level_conv4 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(low_level_conv4)    

	# low_level_conv5
    with tf.variable_scope('low_level_conv5') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 256, 256], stddev=5e-2))
        conv = tf.nn.conv2d(low_level_conv4, weight, [1, 2, 2, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256]))
        pre_activation = tf.nn.bias_add(conv, biases)
        low_level_conv5 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(low_level_conv5)

	# low_level_conv6
    with tf.variable_scope('low_level_conv6') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 256, 512], stddev=5e-2))
        conv = tf.nn.conv2d(low_level_conv5, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        pre_activation = tf.nn.bias_add(conv, biases)
        low_level_conv6 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(low_level_conv6)

	# mid_level_conv1
    with tf.variable_scope('mid_level_conv1') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 512, 512],stddev=5e-2))
        conv = tf.nn.conv2d(low_level_conv6, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        pre_activation = tf.nn.bias_add(conv, biases)
        mid_level_conv1 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(mid_level_conv1)

	# mid_level_conv2
    with tf.variable_scope('mid_level_conv2') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 512, 256], stddev=5e-2))
        conv = tf.nn.conv2d(mid_level_conv1, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[256]))
        pre_activation = tf.nn.bias_add(conv, biases)
        mid_level_conv2 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(mid_level_conv2)

	# global_level_conv1
    with tf.variable_scope('global_level_conv1') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 512, 512], stddev=5e-2))
        conv = tf.nn.conv2d(low_level_conv6, weight, [1, 2, 2, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        pre_activation = tf.nn.bias_add(conv, biases)
        global_level_conv1 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(global_level_conv1)

	# global_level_conv2
    with tf.variable_scope('global_level_conv2') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 512, 512], stddev=5e-2))
        conv = tf.nn.conv2d(global_level_conv1, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        pre_activation = tf.nn.bias_add(conv, biases)
        global_level_conv2 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(global_level_conv2)

	# global_level_conv3
    with tf.variable_scope('global_level_conv3') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 512, 512], stddev=5e-2))
        conv = tf.nn.conv2d(global_level_conv2, weight, [1, 2, 2, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        pre_activation = tf.nn.bias_add(conv, biases)
        global_level_conv3 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(global_level_conv3)

	# global_level_conv4
    with tf.variable_scope('global_level_conv4') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 512, 512], stddev=5e-2))
        conv = tf.nn.conv2d(global_level_conv3, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        pre_activation = tf.nn.bias_add(conv, biases)
        global_level_conv4 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(global_level_conv4)
        global_level_conv4_shape = global_level_conv4.get_shape().as_list()
        
	# global_level_FC1
    with tf.variable_scope('global_level_FC1') as scope:
        #flatten = tf.reshape(conv_5, [conv5_shape[0], conv5_shape[1]*conv5_shape[2]*conv5_shape[3]])
        #print("global level conv4 shape " + str(global_level_conv4_shape))
        flatten = tf.contrib.layers.flatten(global_level_conv4)
        #flatten = tf.reshape(global_level_conv4, [batch_size, global_level_conv4_shape[1] * global_level_conv4_shape[2] * global_level_conv4_shape[3]])
        #print("reshape global level conv4 " + str(flatten.get_shape()))
        dim = flatten.get_shape()[1].value
        weight = tf.Variable(tf.truncated_normal(shape=[dim, 1024], stddev=0.04))
        biases = tf.Variable(tf.constant(0.1, shape=[1024]))
        global_level_FC1 = tf.nn.relu(tf.matmul(flatten, weight) + biases, name=scope.name)
#        _activation_summary(global_level_FC1)
        print("global level FC1 " + str(global_level_FC1.get_shape()))
	
	# global_level_FC2
    with tf.variable_scope('global_level_FC2') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[1024, 512], stddev=0.04))
        biases = tf.Variable(tf.constant(0.1, shape=[512]))
        global_level_FC2 = tf.nn.relu(tf.matmul(global_level_FC1, weight) + biases, name=scope.name)
#        _activation_summary(global_level_FC2)
 #       print("global level FC2 " + str(global_level_FC2.get_shape()))
    
	# global_level_FC3
    with tf.variable_scope('global_level_FC3') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[512, 256], stddev=0.04))
        biases = tf.Variable(tf.constant(0.1, shape=[256]))
        global_level_FC3 = tf.nn.relu(tf.matmul(global_level_FC2, weight) + biases, name=scope.name)
#        _activation_summary(global_level_FC3)
        print("global level FC3 " + str(global_level_FC3.get_shape()))
        
	# fusion_layer
    with tf.variable_scope('fusion_layer') as scope:
        
        global_level_FC3_reshaped = tf.expand_dims(global_level_FC3, 1)
        print("global_level_FC3_reshaped " + str(global_level_FC3_reshaped.get_shape()))
        
        mid_level_conv2_shape = mid_level_conv2.get_shape().as_list()
        mid_level_conv2_reshaped = tf.reshape(mid_level_conv2, [batch_size, mid_level_conv2_shape[1] * mid_level_conv2_shape[2], 256])
        print("mid level conv2 shape reshaped " + str(mid_level_conv2_reshaped.get_shape()))
        mid_level_conv2_reshaped_shape = mid_level_conv2_reshaped.get_shape().as_list()
       # global_level_FC3_tiled = tf.zeros_like(global_level_FC3_reshaped)
        global_level_FC3_tiled = []
        for i in range(global_level_FC3_reshaped.get_shape()[0]):
           # print("in loop " + str(global_level_FC3_reshaped[i].get_shape()))
            global_temp = tf.tile(global_level_FC3_reshaped[i], [mid_level_conv2_reshaped_shape[1], 1])
           # print("globaltemp "  +str(global_temp.get_shape()))
            global_level_FC3_tiled.append(global_temp)
            #global_level_FC3_new[i] = tf.tile(global_level_FC3_reshaped[i], [mid_level_conv2_reshaped.get_shape()[1], mid_level_conv2_reshaped.get_shape()[2]])
            #print("in loop " + str(global_level_FC3_reshaped[i].get_shape())
        global_level_FC3_tiled = tf.stack(global_level_FC3_tiled)
        print("...........global_level_FC3_new " + str(global_level_FC3_tiled.get_shape()))
        
        fusion_level = tf.concat([global_level_FC3_tiled, mid_level_conv2_reshaped], 2)
        print("fusion level " + str(fusion_level.get_shape()))
        
        '''
        print("mid level conv2 shape " + str(mid_level_conv2.get_shape()))
        mid_level_conv2_shape = mid_level_conv2.get_shape().as_list()
        mid_level_conv2_reshaped = tf.reshape(mid_level_conv2, [batch_size, mid_level_conv2_shape[1] * mid_level_conv2_shape[2], 256])
        fusion_level = []
        for j in range(mid_level_conv2_reshaped.shape[0]):
            for i in range(mid_level_conv2_reshaped.shape[1]):
                see_mid = mid_level_conv2_reshaped[j, i, :]
                see_mid_shape = see_mid.get_shape().as_list()
                see_mid = tf.reshape(see_mid, [1, see_mid_shape[0]])
                global_level_FC3_shape = global_level_FC3[j, :].get_shape().as_list()
                see_global = tf.reshape(global_level_FC3[j, :], [1, global_level_FC3_shape[0]])
                fusion = tf.concat([see_mid, see_global], 1)
                fusion_level.append(fusion)
#        print("1 . mid level conv2 reshape " + str(mid_level_conv2_reshaped.get_shape()))
        #mid_level_conv2_reshaped = tf.unstack(mid_level_conv2_reshaped,axis=0)	
        #mp = tf.stack([mid_level_conv2_reshaped])
                
                #print("fusion " + str(fusion.get_shape()))
            
        fusion_level = tf.stack(fusion_level, 1)
        print("fusion level " + str(fusion_level.get_shape()))
        #fusion_level = [tf.concat([see_mid,global_level_FC3],0) for see_mid in mid_level_conv2_reshaped]
        #fusion_level = tf.stack(fusion_level)
        '''
        fusion_level = tf.reshape(fusion_level,[batch_size, 28, 28, 512])	
        weight = tf.Variable(tf.truncated_normal(shape=[1, 1, 512, 256], stddev=5e-2))
        conv = tf.nn.conv2d(fusion_level, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[256]))
        pre_activation = tf.nn.bias_add(conv, biases)
        fusion_layer = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(fusion_layer)
	
	# colorization_level_conv1
    with tf.variable_scope('colorization_level_conv1') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 256, 128], stddev=5e-2))
        conv = tf.nn.conv2d(fusion_layer, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[128]))
        pre_activation = tf.nn.bias_add(conv, biases)
        colorization_level_conv1 = tf.nn.relu(pre_activation, name=scope.name)
        colorization_level_conv1_upsampled = tf.image.resize_images(colorization_level_conv1, [56, 56], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#        _activation_summary(colorization_level_conv1_upsampled)
		
	# colorization_level_conv2
    with tf.variable_scope('colorization_level_conv2') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 64], stddev=5e-2))
        conv = tf.nn.conv2d(colorization_level_conv1_upsampled, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[64]))
        pre_activation = tf.nn.bias_add(conv, biases)
        colorization_level_conv2 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(colorization_level_conv2)

	# colorization_level_conv3
    with tf.variable_scope('colorization_level_conv3') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 64], stddev=5e-2))
        conv = tf.nn.conv2d(colorization_level_conv2, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[64]))
        pre_activation = tf.nn.bias_add(conv, biases)
        colorization_level_conv3 = tf.nn.relu(pre_activation, name=scope.name)
        colorization_level_conv3_upsampled = tf.image.resize_images(colorization_level_conv3, [112, 112], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#        _activation_summary(colorization_level_conv3_upsampled)
                
    # colorization_level_conv4
    with tf.variable_scope('colorization_level_conv4') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 32], stddev=5e-2))
        conv = tf.nn.conv2d(colorization_level_conv3_upsampled, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[32]))
        pre_activation = tf.nn.bias_add(conv, biases)
        colorization_level_conv4 = tf.nn.relu(pre_activation, name=scope.name)
#        _activation_summary(colorization_level_conv4)
                
   # output_layer
    with tf.variable_scope('output_layer') as scope:
        weight = tf.Variable(tf.truncated_normal(shape=[3, 3, 32, 2], stddev=5e-2))
        conv = tf.nn.conv2d(colorization_level_conv4, weight, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.1, shape=[2]))
        pre_activation = tf.nn.bias_add(conv, biases)
        output_layer = tf.nn.sigmoid(pre_activation, name=scope.name)
#        _activation_summary(output_layer)
        print("output_layer "  + str(output_layer.get_shape()))
    return output_layer

In [0]:
def trainNetwork():
    
    #with tf.Graph().as_default():
    #global_step = tf.train.get_or_create_global_step()
    #path = os.path.dirname(os.path.realpath(__file__))
    #path = "drive/app"
    file = "train/"
    dirname = file
    x_img_train, y_img_train = create(dirname)
    x_img_train_shape = x_img_train.shape
    y_img_train_shape = y_img_train.shape
    #print("y train type " + str(type(y_img_train)))
    #print(type(x_img_train))
    #TRAIN SIZE
    train_size = x_img_train_shape[0]
    #print("train data complete")
    
    X_input_train = tf.placeholder(dtype = tf.float32, shape = (batch_size, IMAGE_SIZE, IMAGE_SIZE, 1), name = "input_tensor")
    Y_output_train = tf.placeholder(dtype = tf.float32, shape = (batch_size, IMAGE_SIZE, IMAGE_SIZE, 2), name = "output_tensor")
    
    # X_input_test = tf.placeholder(dtype = data_type(), shape = (FLAGS.batch_size_test, IMAGE_SIZE, IMAGE_SIZE, 1), name = "input_tensor")
    #print(type(x_temp))

    print("............y img train shape " + str(y_img_train_shape))
    '''
      y_img_train_norm = []
    print("y_temp shape " + str(y_temp.shape))
    print("y_temp ith shape " + str(y_temp[0].shape))
    print("1 type y_temp " + str(type(y_temp)))
    
    y_temp_float = tf.cast(y_temp[0] , tf.float32) * (1. / 255)
    print("2 type y_temp_float " + str(type(y_temp_float)))
    print("y_temp float shape " +str(y_temp_float.shape))
    for i in range(y_img_train_shape[0]):
        #print
        y_ = np.array(y_temp[i] , np.float32) * (1. / 255)
        y_img_train_norm.append(y_)
    
    y_img_train_norm = np.array(y_img_train, np.float32) / 255.
    
    
    #print("y_img_train_norm shape " + str(np.shape(y_img_train_norm)))
    #y_img_train_norm[i, :] = tf.cast(y_img_train[i, :] , tf.float32) * (1. / 255)
    #y_img_train_norm = tf.map_fn(lambda image_raw: tf.cast(image_raw , tf.float32) * (1. / 255), y_img_train)
    #print("y norm type " + str(type(y_img_train_norm)))   
    '''
    print("x_img_train_shape " + str(x_img_train_shape))
    logits = create_network(X_input_train)
    print("1............logits shape " +str(logits.shape))
    
    print("logits type " + str(type(logits)))
    #logits_norm = []
    '''
    for i in range(logits.shape[0]):
        logits_ = tf.cast(logits[i, :] , tf.float32) * (1. / 255)
        logits_norm.append(logits_)
    '''
    #logits_norm = tf.map_fn(lambda image_raw : tf.cast(image_raw , tf.float32) * (1. / 255), logits)
    #logits_norm = logits / 255.
    #print("2............logits shape " +str(logits.shape))
    #print("3............logits norm shape " +str(logits_norm.shape))
    #logits_norm = tf.cast(logits , tf.float32) * (1. / 255)
    #logits_norm = tf.image.convert_image_dtype(logits, tf.float32)
    logits_train = tf.image.resize_images(logits, [224, 224], method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print("created")
    print("logits 2 shape " + str(logits_train.get_shape()))
#    color_output = tf.concat((x_temp, logits_train), axis = 3)
    #print("logits size " + str(logits.get_shape()))
    #logits_norm = cv2.normalize(logits, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=-1)
    #TypeError: src is not a numpy array, neither a scalar
    #logits_2 = cv2.resize(logits_norm, (IMAGE_SIZE, IMAGE_SIZE), interpolation = cv2.INTER_CUBIC)
   # x = cv2.resize(x_img[0], (224, 224, 1))
   # x = tf.reshape(x_img[0], [IMAGE_SIZE, IMAGE_SIZE, 1])
    #process test data
    filetest = "test/"
    testdirname = filetest
#   x_img_test, y_img_test = create(testdirname)
    #print("test data complete")
    #logits_test = create_network(X_input_test)
  #  batch = tf.Variable(0, dtype=data_type())
#   loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_output_train, logits=logits_train))
    loss = tf.reduce_mean(tf.square((Y_output_train) - (logits_train)))
    #learning_rate = tf.train.exponential_decay(0.01, batch * FLAGS.batch_size, train_size, 0.95, staircase=True)
    optimizer = tf.train.AdadeltaOptimizer(INITIAL_LEARNING_RATE, rho = 0.95, epsilon = 1e-08).minimize(loss)
    #optimizer = tf.train.GradientDescentOptimizer(1e-4).minimize(loss)
   # print(sess.run(loss, feed_dict = {X_input_train: x_img_train, Y_output_train: y_img_train})) 
    #sess.run(optimizer, feed_dict = {X_input_train: x_img_train, Y_output_train: y_img_train})    
    start_time = time.time()    
    with tf.Session() as sess:
        print("session created")
        init = tf.global_variables_initializer()
        sess.run(init)
        for step in range(int(NUM_EPOCHS * train_size) // batch_size):
          #offset = (step * batch_size) % (train_size - batch_size)
          
          #batch_data = x_temp[offset:(offset + batch_size)]
          #batch_labels = y_img_train_norm[offset:(offset + batch_size)]
          #print("y_img train_norm " + str(y_img_train_norm.shape))
          #print("batch data shape " + str(batch_data.shape))
          #print("batch labels shape " + str(np.shape(batch_labels)))
          #print("logits_train shape " +str(logits_train.shape))
          
          feed_dict = {X_input_train: x_img_train, Y_output_train: y_img_train}
          #print("x temp batch shape " + str(batch_data.shape))
           #print("color output " + str(color_output.shape))
          _, l, predictions = sess.run([optimizer, loss, logits_train], feed_dict=feed_dict)
          color_output = tf.concat((x_img_train, logits_train), axis = 3)
          color_imgs = sess.run(color_output, feed_dict = feed_dict)
          if step % EVAL_FREQUENCY == 0:    
              display_images(color_imgs)
              elapsed_time = time.time() - start_time
              start_time = time.time()
              print('Step %d (epoch %.2f), %.1f ms' %(step, float(step) * batch_size / train_size, 1000 * elapsed_time / EVAL_FREQUENCY))
              print('Minibatch loss: %.3f' % (l))
              #print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels))
              #print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels))
              #print('Validation error: %.1f%%' % error_rate(
              #eval_in_batches(validation_data, sess), validation_labels))
              sys.stdout.flush()
    # Finally print the result!
    #test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
    ##print('Test error: %.1f%%' % test_error)
    #if FLAGS.self_test:
     # print('test_error', test_error)
     # assert test_error == 0.0, 'expected 0.0 test_error, got %.2f' % (test_error,)

    #color_output = []
    #for i in range(FLAGS.batch_size):
   
    #print("optimizer " + str(optimizer))
    
    #logits_test = create_network(X_input_test)
    
    #correct_prediction = tf.equal(tf.argmax(logits), tf.argmax(Y_output))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    #loss = loss(logits, labels)
    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    #train_op = build_model.train(loss, global_step)
    #sess.run(train_op, feed_dict = {})

In [1]:
if __name__ == '__main__':
    #sess = tf.InteractiveSession()
    trainNetwork()
    print("training completed")

NameError: ignored